_**Group Members:**_

## Part 1: The Cost Volume

In [1]:
import urllib.request
import ssl
from os import path as osp
import shutil


def retrieve(file_name, semester='spring24', homework=4):
    if osp.exists(file_name):
        print('Using previously downloaded file {}'.format(file_name))
    else:
        context = ssl._create_unverified_context()
        fmt = 'https://www2.cs.duke.edu/courses/{}/compsci527/homework/{}/{}'
        url = fmt.format(semester, homework, file_name)
        with urllib.request.urlopen(url, context=context) as response:
            with open(file_name, 'wb') as file:
                shutil.copyfileobj(response, file)
        print('Downloaded file {}'.format(file_name))

In [2]:
import skimage.io as io


file_names = ['hand0.pgm', 'hand1.pgm']
for file_name in file_names:
    retrieve(file_name)
frames = [io.imread(file_name) for file_name in file_names]

Using previously downloaded file hand0.pgm
Using previously downloaded file hand1.pgm


### Problem 1.1

## Part 2: Camera Transformations

### Problem 2.1 (Exam Style)

## Part 3: Epipolar Geometry 

### Problem 3.1 (Exam Style)

### Problem 3.2 (Exam Style)

### Problem 3.3 (Exam Style)

## Part 4: Epipolar Lines in Python

### Problem 4.1

In [11]:
import pickle


file_name = 'test_cases.pkl'
retrieve(file_name)
with open(file_name, 'rb') as file:
    test_cases = pickle.load(file)

Using previously downloaded file test_cases.pkl


In [12]:
def camera_frame():
    x_frame = np.array((-1, 1, 1, -1, -1)) / 2
    y_frame = np.array((-1, -1, 1, 1, -1)) / 2
    plt.plot(x_frame, y_frame, lw=1.5, c='gray')
    plt.axhline(0, lw=0.25, c='gray')
    plt.axvline(0, lw=0.25, c='gray')
    plt.axis('square')
    plt.axis('off')
    epsilon = 0.005
    half = 1/2 + epsilon
    plt.xlim(-half, half)
    plt.ylim(half, -half)

### Problem 4.2 (Exam Style)

### Problem 4.3 (Exam Style)